# Econometric UNFCCC Green Cities Commitment Analysis: USA
## Data Preparation

#### Importing and Cleaning Datasets
Data includes the following:
1. Full list of US City, County and State data
2. UNFCCC data on cities' commitments, actions, etc
3. 2020 Election results by county
4. Land Temperature data
5. US Bureau of Labor Statistics State Unemployment Rate
6. Poverty Estimates for U.S., States, and Counties, 2021
7. Number of Natural Disasters by State since 1953
8. State-level Energy-Related Emissions Per Capita 2026-2021

In [1]:
import pandas as pd
import numpy as np

##### 1. US Cities, Counties and State Data
simplemaps.com. US Cities Database | Simplemaps.com. [online] Available at: https://simplemaps.com/data/us-cities.

In [2]:
USACities = pd.read_csv('RawData/USACities.csv')

In [3]:
USACities = USACities[USACities['population'] >= 1000]
USACities['UNFCCC'] = False

In [4]:
USACities.head()

,city,state,state_name,fips,county,latitude,longitude,population,UNFCCC
0,New York City,NY,New York,36081,Queens,40.6943,-73.9249,18908608,False
1,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,False
2,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8497759,False
3,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6080145,False
4,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5970127,False


##### 2. UNFCCC Data
Global Climate Action UNFCCC - Actor Tracking (2022). Available at: https://climateaction.unfccc.int/Actors.

In [5]:
UNFCCC = pd.read_csv('RawData/UNFCCC.csv')

In [6]:
UNFCCC = UNFCCC[(UNFCCC['country'] == 'United States of America') & (UNFCCC['organizationType'] == 'City') & (UNFCCC['actorProperties_population'] >= 1000)]  # Filter for cities in the US with populations over 1000 
UNFCCC['Date'] = pd.to_datetime(UNFCCC['Date'], format = '%d/%m/%Y')  # format date as datetime object
UNFCCC = UNFCCC.sort_values(by = 'Date', ascending = False)
UNFCCC = UNFCCC.drop_duplicates(subset = 'publicId', keep = 'first')  # keep only most recent observation of a city
UNFCCC.reset_index(inplace = True)  # Reset index
UNFCCC.drop(['index','Date','id','accountingYear','organizationType'], axis = 1, inplace = True)  # drop unneeded columns

In [7]:
# Generate City and State Columns from OrganisationName
UNFCCC['city'] = UNFCCC['organizationName'].apply(lambda x: x.split(',')[0].strip())
UNFCCC['state'] = UNFCCC['organizationName'].apply(lambda x: x.split(',')[1].strip() if len(x.split(',')) == 2 else np.NaN)   

# Clean City names for better joining
strings_to_remove = ['City of', 'City Of', 'Town of', 'Town Of', 'Township Of', 'City and County of', 'Borough Of', 'Village of', '(Town)']
for string in strings_to_remove:
    UNFCCC['city'] = UNFCCC['city'].str.replace(string, '', regex = False)

UNFCCC['city'] = UNFCCC['city'].apply(lambda x: x.strip())

UNFCCC = UNFCCC[['city','state','hasCommitments','hasActionsUndertaken',
                'hasEmissionInventory','hasInitiativeParticipations','hasImpact','hasMitigations',
                'hasAdaptations','hasRiskAssessments','hasClimateActionPlans','hasFinanceActions']]

Remove the following irregular / nonconforming observations:
1. Metropolitan Council, Twin Cities: covers bistate area and Minneapolis is already included
2. Chicago Metropolitan Mayors Caucus:  Chicago is already included
3. Metropolitan Washington Council of Governments (COG):  covers bistate area and District of Columbia is already included
4. Mid-America Regional Council:  covers bistate area and Kansas City is already included
5. San Francisco/Bay Area Air Quality Management District:  San Francisco is already included

In [8]:
cities_to_remove = ['Metropolitan Council', 'Chicago Metropolitan Mayors Caucus', 'Metropolitan Washington Council of Governments (COG)', 'Mid-America Regional Council', 'San Francisco/Bay Area Air Quality Management District’S']
UNFCCC = UNFCCC[~UNFCCC['city'].isin(cities_to_remove)]

Make the following alterations to irregular / nonconforming observations:
1. Metropolitan Government of Nashville and Davidson County, TN: rename to Nashville
2. Durham: add North Carolina as state

In [9]:
UNFCCC.loc[UNFCCC['city'] == 'Metropolitan Government of Nashville and Davidson County', 'city'] = 'Nashville'
UNFCCC.loc[UNFCCC['city'] == 'Durham', 'state'] = 'NC'

In [10]:
UNFCCC.head()

,city,state,hasCommitments,hasActionsUndertaken,hasEmissionInventory,hasInitiativeParticipations,hasImpact,hasMitigations,hasAdaptations,hasRiskAssessments,hasClimateActionPlans,hasFinanceActions
0,Park Forest,IL,True,True,False,False,False,True,True,True,True,False
1,Emeryville,CA,True,True,True,True,False,True,True,True,True,False
2,Grand Rapids,MI,True,True,False,True,False,True,True,True,False,False
3,Fremont,CA,True,True,True,True,False,True,True,True,True,False
4,Fort Worth,TX,False,True,False,False,False,False,True,False,False,False


##### 3. 2020 Election Results by County and by State
MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2020", https://doi.org/10.7910/DVN/VOQCHQ, Harvard Dataverse, V12

Wikipedia Contributors (2019). 2020 United States presidential election. [online] Wikipedia. Available at: https://en.wikipedia.org/wiki/2020_United_States_presidential_election.

In [11]:
ElectionbyCounty = pd.read_csv('RawData/ElectionbyCounty.csv')

In [12]:
ElectionbyCounty['fips'] = ElectionbyCounty['fips'].astype(int)  # convert to integer for later joins

ElectionbyCounty = ElectionbyCounty.pivot(index = ['state_name','state','county','fips','totalvotes'], columns = 'party', values = 'candidatevotes').reset_index()  # pivot for widened form

RedCounty = lambda row: False if row['DEMOCRAT'] > row['REPUBLICAN'] else True  # Assign true if county had most votes for republican presidential candidate
ElectionbyCounty['redCounty'] = ElectionbyCounty.apply(RedCounty, axis = 1)
ElectionbyCounty = ElectionbyCounty[['state','fips','redCounty']]

In [13]:
ElectionbyCounty.head()

party,state,fips,redCounty
0,AL,1001,True
1,AL,1003,True
2,AL,1005,True
3,AL,1007,True
4,AL,1009,True


In [14]:
ElectionbyState = pd.read_csv('RawData/ElectionbyState.csv')

In [15]:
ElectionbyState['winner'] = ElectionbyState.iloc[:,1:].idxmax(axis = 1)
RedState = lambda row: False if row['winner'] == 'Biden/HarrisDemocratic' else True
ElectionbyState['redState'] = ElectionbyState.apply(RedState, axis = 1)
ElectionbyState = ElectionbyState[['state','redState']]

In [16]:
ElectionbyState.head()

,state,redState
0,AL,True
1,AK,True
2,AZ,False
3,AR,True
4,CA,False


##### 4. Land Temperatures from 1828 to 2013
www.kaggle.com. (Berkeley Earth). Climate Change: Earth Surface Temperature Data. [online] Available at: https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data.

In [17]:
LandTemp = pd.read_csv('RawData/GlobalLandTemperatures.csv')

In [18]:
LandTemp = LandTemp[LandTemp['Country'] == 'United States']  # filter for USA State Data
LandTemp['Date'] = pd.to_datetime(LandTemp['dt'], format = '%Y-%m-%d')  # create datetime column
LandTemp['Year'] = LandTemp['Date'].dt.year  # create year column
LandTemp['Month'] = LandTemp['Date'].dt.month_name()  # create month column
LandTemp = LandTemp[['Year','Month','State','AverageTemperature']]

In [19]:
LandTemp = LandTemp[LandTemp['Month'].isin(['January','July'])]  # keep only January and July observations
LandTemp = LandTemp.pivot(index = ['Year','State'], columns = 'Month', values = 'AverageTemperature').reset_index()  # Pivot to get widened form
LandTemp['janJulyDiff'] = LandTemp['July'] - LandTemp['January']  # calculate July to January temperature difference
LandTemp.reset_index(inplace = True)
LandTemp.dropna(inplace = True)

In [20]:
# large changes in temperature --> https://www.climate.gov/news-features/understanding-climate/climate-change-global-temperature
# Indicator: difference in averages of 1940-1960 and 2000-2020
LandTemp20002013 = LandTemp[LandTemp['Year'].between(2000,2003)].groupby('State')['janJulyDiff'].mean()  # mean of January July temperature differences 2000-2013
LandTemp19401960 = LandTemp[LandTemp['Year'].between(1940,1960)].groupby('State')['janJulyDiff'].mean()  # mean of January July temperature differences 1940-1960

tempDiff = LandTemp20002013 - LandTemp19401960
tempDiff = pd.DataFrame(tempDiff).reset_index()
tempDiff.rename(columns = {'janJulyDiff':'tempDiff'}, inplace = True)

In [21]:
tempDiff.head()

,State,tempDiff
0,Alabama,1.679440
1,Alaska,-3.913524
2,Arizona,-0.513298
3,Arkansas,0.912119
4,California,-0.821321


##### 5. US Bureau of Labor Statistics State Unemployment Rate by County 2022
Unemployment: U.S. Department of Labor, Bureau of Labor Statistics, Local Area Unemployment Statistics (LAUS)

In [22]:
Unemp = pd.read_csv('RawData/UnemploymentbyCounty.csv')

In [23]:
Unemp = Unemp[['fips','state','unemploymentRate']]

In [24]:
Unemp.head()

,fips,state,unemploymentRate
0,0,US,0.037
1,1000,AL,0.026
2,1001,AL,0.023
3,1003,AL,0.024
4,1005,AL,0.041


##### 6. Poverty Estimates for U.S., States, and Counties, 2021
U.S. Department of Commerce, Bureau of the Census, Small Area Income and Poverty Estimates (SAIPE) Program

In [25]:
Poverty = pd.read_csv('RawData/PovertyEstimates.csv')

In [26]:
Poverty['povertyProp'] = Poverty['PCTPOVALL_2021'] / 100  # convert to decimal
Poverty = Poverty[['fips','povertyProp']]  # option to include: 'county','state',

In [27]:
Poverty.head()

,fips,povertyProp
0,0,0.128
1,1000,0.163
2,1001,0.107
3,1003,0.108
4,1005,0.230


##### 7. Number of Natural Disasters by State since 1953
worldpopulationreview.com. Natural Disasters by State [Updated May 2023]. [online] Available at: https://worldpopulationreview.com/state-rankings/natural-disasters-by-state.

In [28]:
NaturalDisasters = pd.read_csv('RawData/NaturalDisasters.csv')

In [29]:
NaturalDisasters.head()

,state,numDisasters
0,AL,82
1,AK,55
2,AZ,68
3,AR,71
4,CA,284


##### 8. State-level Energy-Related Emissions Per Capita 2026-2021
U.S. Energy Information Administration, State Energy Data System and EIA calculations. Available at: https://www.eia.gov/environment/emissions/state/													

In [30]:
Emissions = pd.read_csv('RawData/StateEmissions.csv')

In [31]:
Emissions['avgEmissionsPerCapita'] = Emissions.iloc[:,1:].mean(axis = 1)  # get average of emissions per capita
Emissions.rename(columns = {'State':'state'}, inplace = True)
Emissions = Emissions[['state','avgEmissionsPerCapita']]

In [32]:
Emissions.head()

,state,avgEmissionsPerCapita
0,AL,21.900000
1,AK,47.683333
2,AZ,12.383333
3,AR,20.983333
4,CA,8.716667


#### 9. County-level Percent of adults with less than a high school diploma, 2017-21
USDA, Economic Research Service using data from U.S. Department of Commerce, Bureau of the Census, 1970, 1980, 1990, 2000 Censuses of Population, and 2008–12 and 2017–21 American Community Survey 5-year period county-level estimates. Data as of January 24, 2023.

In [33]:
EducationbyCounty = pd.read_excel('RawData/EducationbyCounty.xlsx')

In [34]:
EducationbyCounty = EducationbyCounty[['fips','state','lessThanHighSchoolProp']]

In [35]:
EducationbyCounty.head()

,fips,state,lessThanHighSchoolProp
0,0,US,0.111260
1,1000,AL,0.125973
2,1001,AL,0.104155
3,1003,AL,0.089858
4,1005,AL,0.243290


#### 10. Median Household Income by County, 2021
U.S. Department of Commerce, Bureau of the Census, Small Area Income and Poverty Estimates (SAIPE) Program.

In [36]:
HouseholdIncome = pd.read_csv('RawData/HouseholdIncomebyCounty.csv')

In [37]:
HouseholdIncome.head()

,fips,state,medianHouseholdIncome
0,0,US,69717.0
1,1000,AL,53990.0
2,1001,AL,66444.0
3,1003,AL,65658.0
4,1005,AL,38649.0


#### Joining Datasets

1. Add additional variables to the All Cities dataset

In [38]:
df = pd.merge(USACities, ElectionbyCounty, how = 'left', on = ['state','fips'])
df = pd.merge(df, ElectionbyState, how = 'left', on = 'state')
df = pd.merge(df, tempDiff, how = 'left', left_on = 'state_name', right_on = 'State')
df = pd.merge(df, Unemp, on = ['state','fips'], how = 'left')
df = pd.merge(df, Poverty, on = 'fips', how = 'left')
df = pd.merge(df, NaturalDisasters, on = 'state', how = 'left')
df = pd.merge(df, Emissions, on = 'state', how = 'left')
df = pd.merge(df, EducationbyCounty, on = ['state','fips'], how = 'left')
df = pd.merge(df, HouseholdIncome, on = ['state','fips'], how = 'left')

In [39]:
df.head()

,city,state,state_name,fips,county,latitude,longitude,population,UNFCCC,redCounty,redState,State,tempDiff,unemploymentRate,povertyProp,numDisasters,avgEmissionsPerCapita,lessThanHighSchoolProp,medianHouseholdIncome
0,New York City,NY,New York,36081,Queens,40.6943,-73.9249,18908608,False,False,False,New York,-0.930190,0.052,0.136,95.0,8.316667,0.172983,72807.0
1,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11922389,False,False,False,California,-0.821321,0.049,0.141,284.0,8.716667,0.199572,77356.0
2,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8497759,False,False,False,Illinois,0.017488,0.050,0.138,60.0,15.483333,0.120266,72063.0
3,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6080145,False,False,True,Florida,1.829571,0.026,0.152,130.0,10.883333,0.175103,58905.0
4,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,5970127,False,False,True,Texas,-0.085524,0.042,0.164,255.0,23.050000,0.181317,63539.0


2. Create dataset of all cities in the US with a flag for whether they have signed up for the UNFCCC

In [40]:
AllCities = pd.merge(df, UNFCCC, how = 'left', on = ['state','city'])

In [41]:
# Create UNFCCC flag
unfcc_flag = lambda x: False if np.isnan(x) else True
AllCities['UNFCCC'] = AllCities['hasCommitments'].apply(unfcc_flag)

bool_columns = ['redState','redCounty','hasCommitments','hasActionsUndertaken','hasEmissionInventory','hasInitiativeParticipations','hasImpact','hasMitigations','hasAdaptations', 'hasRiskAssessments', 'hasClimateActionPlans','hasFinanceActions']
AllCities[bool_columns] = AllCities[bool_columns].fillna(False).applymap(pd.to_numeric, errors='coerce').astype(bool)  # assign False to NaNs and ensure type bool

# Reorder and keep only necessary columns
AllCities = AllCities[['city','county','state','fips','latitude','longitude','population','redCounty','redState','unemploymentRate','povertyProp','tempDiff','numDisasters','avgEmissionsPerCapita','lessThanHighSchoolProp','medianHouseholdIncome',
                       'UNFCCC','hasCommitments','hasActionsUndertaken','hasEmissionInventory','hasInitiativeParticipations','hasImpact','hasMitigations','hasAdaptations','hasRiskAssessments','hasClimateActionPlans', 'hasFinanceActions']]

In [42]:
AllCities.head()

,city,county,state,fips,latitude,longitude,population,redCounty,redState,unemploymentRate,...,hasCommitments,hasActionsUndertaken,hasEmissionInventory,hasInitiativeParticipations,hasImpact,hasMitigations,hasAdaptations,hasRiskAssessments,hasClimateActionPlans,hasFinanceActions
0,New York City,Queens,NY,36081,40.6943,-73.9249,18908608,False,False,0.052,...,True,True,True,True,False,True,True,True,True,False
1,Los Angeles,Los Angeles,CA,6037,34.1141,-118.4068,11922389,False,False,0.049,...,True,True,True,True,False,True,True,True,True,True
2,Chicago,Cook,IL,17031,41.8375,-87.6866,8497759,False,False,0.050,...,True,True,True,True,False,True,True,True,True,True
3,Miami,Miami-Dade,FL,12086,25.7840,-80.2101,6080145,False,True,0.026,...,True,True,True,True,False,True,True,True,True,False
4,Houston,Harris,TX,48201,29.7860,-95.3885,5970127,False,True,0.042,...,True,True,True,True,False,True,True,True,True,False


In [43]:
AllCities.to_csv('allcities.csv', index = False)

3. Create dataset with only cities which have signed up for the UNFCCC with all the additional variables

In [44]:
UNFCCC = pd.merge(UNFCCC, df, how = 'left', on = ['state','city'])

UNFCCC = UNFCCC[['city','county','state','fips','latitude','longitude','population','redCounty','redState','unemploymentRate','povertyProp','tempDiff','numDisasters','avgEmissionsPerCapita','lessThanHighSchoolProp','medianHouseholdIncome',
                 'UNFCCC','hasCommitments','hasActionsUndertaken','hasEmissionInventory','hasInitiativeParticipations', 'hasImpact','hasMitigations', 'hasAdaptations', 'hasRiskAssessments','hasClimateActionPlans','hasFinanceActions']]

In [45]:
UNFCCC.head()

,city,county,state,fips,latitude,longitude,population,redCounty,redState,unemploymentRate,...,hasCommitments,hasActionsUndertaken,hasEmissionInventory,hasInitiativeParticipations,hasImpact,hasMitigations,hasAdaptations,hasRiskAssessments,hasClimateActionPlans,hasFinanceActions
0,Park Forest,Cook,IL,17031.0,41.4817,-87.6867,21042.0,False,False,0.050,...,True,True,False,False,False,True,True,True,True,False
1,Emeryville,Alameda,CA,6001.0,37.8382,-122.2932,12840.0,False,False,0.033,...,True,True,True,True,False,True,True,True,True,False
2,Grand Rapids,Kent,MI,26081.0,42.9619,-85.6562,609023.0,False,False,0.035,...,True,True,False,True,False,True,True,True,False,False
3,Fremont,Alameda,CA,6001.0,37.5265,-121.9843,228795.0,False,False,0.033,...,True,True,True,True,False,True,True,True,True,False
4,Fort Worth,Tarrant,TX,48439.0,32.7817,-97.3474,924663.0,False,True,0.036,...,False,True,False,False,False,False,True,False,False,False


In [46]:
UNFCCC.to_csv('UNFCCC.csv', index = False)